<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#相关变量分析" data-toc-modified-id="相关变量分析-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>相关变量分析</a></span><ul class="toc-item"><li><span><a href="#比赛介绍" data-toc-modified-id="比赛介绍-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>比赛介绍</a></span></li><li><span><a href="#相关有趣的知识点" data-toc-modified-id="相关有趣的知识点-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>相关有趣的知识点</a></span></li><li><span><a href="#读入数据" data-toc-modified-id="读入数据-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>读入数据</a></span><ul class="toc-item"><li><span><a href="#Target-variable" data-toc-modified-id="Target-variable-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Target variable</a></span></li><li><span><a href="#full-Visitor-ld" data-toc-modified-id="full-Visitor-ld-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>full Visitor ld</a></span></li></ul></li><li><span><a href="#基本数据分析" data-toc-modified-id="基本数据分析-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>基本数据分析</a></span><ul class="toc-item"><li><span><a href="#目标变量" data-toc-modified-id="目标变量-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>目标变量</a></span></li><li><span><a href="#设备与-target-的关系" data-toc-modified-id="设备与-target-的关系-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>设备与 target 的关系</a></span></li><li><span><a href="#target-当成时间序列" data-toc-modified-id="target-当成时间序列-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>target 当成时间序列</a></span></li><li><span><a href="#geographic-与-target-关系" data-toc-modified-id="geographic-与-target-关系-1.4.4"><span class="toc-item-num">1.4.4&nbsp;&nbsp;</span>geographic 与 target 关系</a></span></li><li><span><a href="#traffic-source-和-target-的关系" data-toc-modified-id="traffic-source-和-target-的关系-1.4.5"><span class="toc-item-num">1.4.5&nbsp;&nbsp;</span>traffic source 和 target 的关系</a></span></li></ul></li></ul></li><li><span><a href="#参考资料" data-toc-modified-id="参考资料-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>参考资料</a></span></li></ul></div>

# 相关变量分析

*By Jared Gong*

本文档专注于分析各个变量。

## 比赛介绍

>> Google Analytics Customer Revenue Prediction 

**Objective of the competition:**

在本次竞赛中，我们面临的挑战是分析Google Merchandise Store（也称为GStore，销售谷歌周边）客户数据集，predicting the log of the total revenue per user, be aware that not all rows in `test.csv` will correspond to a row in the submission, but all unique `fullVisitorIds` will correspond to a row in the submission.


有多个列包含不同深度的JSON blob。在其中一个JSON列（`total`）中，子列 `transactionRevenue` 包含我们尝试预测的收入信息。该子列仅存在于训练数据中。具体的来说，我们预测每个用户的所有交易总和的自然对数。对于测试集中的每个用户，目标是：

$$
\text{target}^\text{user} = \ln \sum_{i=1}^n \text{transaction}_i^\text{user}
$$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 相关有趣的知识点

## 读入数据

本数据有多个列包含不同深度的JSON blob。


关于数据集：

与大多数其他的kaggle比赛类似，我们给出了两个数据集

    train.csv
    test.csv

数据集中的每一行都是对商店的一次访问。我们预测每个用户的所有交易总和的自然对数。

需要注意的是，有多个列包含不同深度的JSON blob。在其中一个JSON列（`total`）中，子列 `transactionRevenue` 包含我们尝试预测的收入信息。

In [ ]:
import os
import json
from pandas.io.json import json_normalize

def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, 
                     nrows=nrows) # Important!!
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("../input"))

train = load_df(nrows=100000)
test =load_df('../input/test.csv', nrows=100000)
train.head()


总共12个变量：

1. `channelGrouping`  - 用户通过其进入商店的渠道。
1. `date` - 用户访问Store的日期
1. `device` - 用户访问 Store 的设备类型
1. `fullVisitorId` - Google Merchandise Store的每个用户的唯一标识符。
1. `geoNetwork`  - 包含有关用户地理位置的信息。
1. `sessionId` - 此次访问商店的唯一标识符。
1. `socialEngagementType` - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
1. `totals` - This section contains aggregate values across the session.
1. `trafficSource` - This section contains information about the Traffic Source from which the session originated.
1. `visitId`  - 此会话的标识符。这是通常存储为_utmb cookie的值的一部分。这仅对用户而言是唯一的。对于完全唯一的ID，您应该使用fullVisitorId和visitId的组合。
1. `visitNumber`  - 此用户的会话号。如果这是第一个会话，则将其设置为1。
1. `visitStartTime`  - 时间戳（表示为POSIX时间）。

其中的四个变量 `device, geoNetwork, totals, trafficSource` 进一步处理成多个变量。

最后分解成54个特征, 包括 'channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.browser', 'device.browserSize', 'device.browserVersion',
       'device.deviceCategory', 'device.flashVersion', 'device.isMobile',
       'device.language', 'device.mobileDeviceBranding',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.screenColors', 'device.screenResolution', 'geoNetwork.city',
       'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue', 'totals.visits', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.criteriaParameters',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source' 

###  Target variable


我们预测每个用户的所有交易总和的自然对数。对于测试集中的每个用户，目标是：

$$
\text{target}^\text{user} = \ln \sum_{i=1}^n \text{transaction}_i^\text{user}
$$

看看进行散点图。

In [ ]:
train["totals.transactionRevenue"] = train["totals.transactionRevenue"].astype('float')
gdf = train.groupby("fullVisitorId")["totals.transactionRevenue"].sum().reset_index()

plt.figure(figsize=(6,4))
plt.scatter(range(gdf.shape[0]), np.sort(np.log1p(gdf["totals.transactionRevenue"].values)))
plt.xlabel('index', fontsize=12)
plt.ylabel('TransactionRevenue', fontsize=12)
plt.show()

In [ ]:
gdf.describe()

从中可以看出一个重要的点：

*The 80/20 rule has proven true for many businesses–only a small percentage of customers produce most of the revenue. As such, marketing teams are challenged to make appropriate investments in promotional strategies.*

### full Visitor ld

与客户 Id 相关的变量包括 `fullVisitorId, sessionId, visitId, visitNumber`，一个 `fullVisitorId` 包含多个 `sessionId`, 而一个 `sessionId` 唯一对应一个 `visitId`，个数和 `visitNumber` 相关。

> 我们需要创建一个以 `fullVisitorId` 为 index 的数据框。

## 基本数据分析

In [ ]:
zeroColumns = [c for c in train.columns if train[c].nunique() == 1]
zeroColumns
# 数据中方差不为0的特征

### 目标变量

In [ ]:
nzi = pd.notnull(train["totals.transactionRevenue"]).sum()
nzr = (gdf["totals.transactionRevenue"] > 0).sum()
print("Nusmber of instances in train set with non-zero revenue : ", nzi, " and ratio is : ", nzi / train.shape[0])
print("Number of unique customers with non-zero revenue : ", nzr, "and the ratio is : ", nzr / gdf.shape[0])


访客人数和共同访客人数：

现在让我们看一下训练和测试集中的不同访客数量，以及共同的访客的数量。

In [ ]:
print("Number of unique visitors in train set : ",train.fullVisitorId.nunique(), " out of rows : ",train.shape[0])
print("Number of unique visitors in test set : ",test.fullVisitorId.nunique(), " out of rows : ",test.shape[0])
print("Number of common visitors in train and test set : ",len(set(train.fullVisitorId.unique()).intersection(set(test.fullVisitorId.unique())) ))

### 设备与 target 的关系

与设备相关的三个重要属性是：

- 浏览器(chrome,firefox, IE)
- 操作系统（mac, linux, windows）
- 设备类型（移动端，桌面端和平板）

我们考察这三个变量和目标变量的关系。

In [ ]:
import seaborn as sns
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

def horizontal_bar_chart(cnt_srs, color):
    trace = go.Bar(
        y=cnt_srs.index[::-1],
        x=cnt_srs.values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

In [ ]:
# Device Browser
cnt_srs = train.groupby('device.browser')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(50, 171, 96, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(50, 171, 96, 0.6)')

# Device Category
cnt_srs = train.groupby('device.deviceCategory')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace3 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(71, 58, 131, 0.8)')
trace4 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(71, 58, 131, 0.8)')

# Operating system
cnt_srs = train.groupby('device.operatingSystem')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace5 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(246, 78, 139, 0.6)')
trace6 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10),'rgba(246, 78, 139, 0.6)')

# Creating two subplots
fig = tools.make_subplots(rows=3, cols=2, vertical_spacing=0.04, 
                          subplot_titles=["Device Browser - Count", "Device Browser - Non-zero Revenue Count",
                                          "Device Category - Count",  "Device Category - Non-zero Revenue Count",
                                          "Device Operating System - Count", "Device Operating System - Non-zero Revenue Count"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace5, 3, 1)
fig.append_trace(trace6, 3, 2)

fig['layout'].update(height=1200, width=800, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
py.iplot(fig, filename='device-plots')


推论：

- 设备浏览器分布在非零收入图的计数和计数方面看起来相似
- 在设备类别方面，与移动设备相比，桌面似乎具有更高的非零收入计数百分比。
- 在设备操作系统中，虽然计数的次数多来自Windows，但收入不为零的计数数量对于Macintosh来说更多。
- Chrome操作系统的非零收入占比也较高
- 在移动操作系统方面，与Android相比，iOS拥有更多的非零收入计数

### target 当成时间序列



In [ ]:
import datetime

def scatter_plot(cnt_srs, color):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        showlegend=False,
        marker=dict(
            color=color,
        ),
    )
    return trace

train['date'] = train['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
cnt_srs = train.groupby('date')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_index()
#cnt_srs.index = cnt_srs.index.astype('str')
trace1 = scatter_plot(cnt_srs["count"], 'red')
trace2 = scatter_plot(cnt_srs["count of non-zero revenue"], 'blue')

fig = tools.make_subplots(rows=2, cols=1, vertical_spacing=0.08,
                          subplot_titles=["Date - Count", "Date - Non-zero Revenue count"])
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig['layout'].update(height=800, width=800, paper_bgcolor='rgb(233,233,233)', title="Date Plots")
py.iplot(fig, filename='date-plots')


推论：

- 我们的训练数据集中包含2016年8月1日至2017年7月31日的数据
- 2016年11月，虽然访客人数有所增加，但在此期间非零收入计数没有增加。

In [ ]:
test['date'] = test['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
cnt_srs = test.groupby('date')['fullVisitorId'].size()


trace = scatter_plot(cnt_srs, 'red')

layout = go.Layout(
    height=400,
    width=800,
    paper_bgcolor='rgb(233,233,233)',
    title='Dates in Test set'
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")


在测试集中，我们的日期为2017年8月2日至2018年4月30日。因此，训练和测试集之间没有共同日期。因此，对此数据集进行基于时间的验证可能是个好主意。

### geographic 与 target 关系

In [ ]:
# Geographic Information:

# Continent
cnt_srs = train.groupby('geoNetwork.continent')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(58, 71, 80, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(58, 71, 80, 0.6)')

# Sub-continent
cnt_srs = train.groupby('geoNetwork.subContinent')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace3 = horizontal_bar_chart(cnt_srs["count"], 'orange')
trace4 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'orange')

# Network domain
cnt_srs = train.groupby('geoNetwork.networkDomain')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace5 = horizontal_bar_chart(cnt_srs["count"].head(10), 'blue')
trace6 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'blue')

# Creating two subplots
fig = tools.make_subplots(rows=3, cols=2, vertical_spacing=0.08, horizontal_spacing=0.15, 
                          subplot_titles=["Continent - Count", "Continent - Non-zero Revenue Count",
                                          "Sub Continent - Count",  "Sub Continent - Non-zero Revenue Count",
                                          "Network Domain - Count", "Network Domain - Non-zero Revenue Count"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace5, 3, 1)
fig.append_trace(trace6, 3, 2)

fig['layout'].update(height=1200, width=800, paper_bgcolor='rgb(233,233,233)', title="Geography Plots")
py.iplot(fig, filename='geo-plots')



推论：

- 在非洲大陆的情节中，我们可以看到，美国的计数数量更多，而且收入非零的计数最多
- 虽然亚洲和欧洲的计数很多，但来自这些大陆的非零收入数量相对较低。
- 我们也可以从次大陆的图推断前两点。
- 如果网络域是“unknown.unknown”而不是“（未设置）”，则具有非零收入的计数数量往往较低。

### traffic source 和 target 的关系

In [ ]:
cnt_srs = train.groupby('trafficSource.source')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'green')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'green')

# Sub-continent
cnt_srs = train.groupby('trafficSource.medium')['totals.transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace3 = horizontal_bar_chart(cnt_srs["count"], 'purple')
trace4 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'purple')

# Creating two subplots
fig = tools.make_subplots(rows=2, cols=2, vertical_spacing=0.08, horizontal_spacing=0.15, 
                          subplot_titles=["Traffic Source - Count", "Traffic Source - Non-zero Revenue Count",
                                          "Traffic Source Medium - Count",  "Traffic Source Medium - Non-zero Revenue Count"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(height=1000, width=800, paper_bgcolor='rgb(233,233,233)', title="Traffic Source Plots")
py.iplot(fig, filename='traffic-source-plots')


推论：

- 在流量来源图中，尽管Youtube在数据集中具有大量计数，但非零收入计数的数量非常少。
- Google plex的非零收入计数与流量来源图中的总计数之比很高。
- 在流量来源媒体上，与“organic”媒体相比，“referral”具有更多的非零收入计数。

# 参考资料

- Simple Exploration+Baseline - GA Customer Revenue
 https://www.kaggle.com/sudalairajkumar/simple-exploration-baseline-ga-customer-revenue